# CHAPTER 7 Data Cleaning and Preparation

其实数据分析中80%的时间都是在数据清理部分，loading, clearning, transforming, rearranging。而pandas非常适合用来执行这些任务。

# 7.1 Handling Missing Data

在pandas中，missing data呈现的方式有些缺点的，但对大部分用户能起到足够的效果。对于数值型数据，pandas用浮点值Nan(Not a Number)来表示缺失值。我们称之为识别符（sentinel value)，这种值能被轻易检测到：

In [1]:
import pandas as pd
import numpy as np

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [4]:
string_data[0] = None

In [5]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

这里有一些用来处理缺失值的函数：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/zq0q8.png)

# 1 Filtering Out Missing Data（过滤缺失值）

有一些方法来过滤缺失值。可以使用pandas.isnull和boolean indexing, 配合使用dropna。对于series，只会返回non-null数据和index values:

In [6]:
from numpy import nan as NA

In [7]:
data = pd.Series([1, NA, 3.5, NA, 7])

In [8]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

上面的等同于：

In [9]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

对于DataFrame，会复杂一些。你可能想要删除包含有NA的row和column。dropna默认会删除包含有缺失值的row：

In [10]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [11]:
cleaned = data.dropna()

In [12]:
cleaned

,0,1,2
0,1.0,6.5,3.0


设定`how=all`只会删除那些全是NA的行：

In [13]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


删除列也一样，设置axis=1:

In [14]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [15]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


一种删除DataFrame row的相关应用是是time series data。假设你想要保留有特定数字的观测结果，可以使用thresh参数：

In [16]:
df = pd.DataFrame(np.random.randn(7, 3))

In [17]:
df

,0,1,2
0,-1.169477,-0.140065,0.167299
1,-1.000001,0.588144,0.338448
2,0.360180,1.845757,0.157768
3,0.908547,-1.623989,-1.390020
4,-0.395955,0.531427,1.157834
5,-0.276360,-0.240133,1.478797
6,-0.061835,1.676122,0.856263


In [18]:
df.iloc[:4, 1] = NA
df

,0,1,2
0,-1.169477,NaN,0.167299
1,-1.000001,NaN,0.338448
2,0.360180,NaN,0.157768
3,0.908547,NaN,-1.390020
4,-0.395955,0.531427,1.157834
5,-0.276360,-0.240133,1.478797
6,-0.061835,1.676122,0.856263


In [19]:
df.iloc[:2, 2] = NA
df

,0,1,2
0,-1.169477,NaN,NaN
1,-1.000001,NaN,NaN
2,0.360180,NaN,0.157768
3,0.908547,NaN,-1.390020
4,-0.395955,0.531427,1.157834
5,-0.276360,-0.240133,1.478797
6,-0.061835,1.676122,0.856263


In [20]:
df.dropna()

,0,1,2
4,-0.395955,0.531427,1.157834
5,-0.276360,-0.240133,1.478797
6,-0.061835,1.676122,0.856263


In [21]:
df.dropna(thresh=2) 

,0,1,2
2,0.360180,NaN,0.157768
3,0.908547,NaN,-1.390020
4,-0.395955,0.531427,1.157834
5,-0.276360,-0.240133,1.478797
6,-0.061835,1.676122,0.856263


# 2 Filling In Missing Data（填补缺失值）

不是删除缺失值，而是用一些数字填补。对于大部分目的，fillna是可以用的。调用fillna的时候设置好一个常用用来替换缺失值：

In [22]:
df.fillna(0)

,0,1,2
0,-1.169477,0.000000,0.000000
1,-1.000001,0.000000,0.000000
2,0.360180,0.000000,0.157768
3,0.908547,0.000000,-1.390020
4,-0.395955,0.531427,1.157834
5,-0.276360,-0.240133,1.478797
6,-0.061835,1.676122,0.856263


给fillna传入一个dict，可以给不同列替换不同的值：

In [23]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-1.169477,0.500000,0.000000
1,-1.000001,0.500000,0.000000
2,0.360180,0.500000,0.157768
3,0.908547,0.500000,-1.390020
4,-0.395955,0.531427,1.157834
5,-0.276360,-0.240133,1.478797
6,-0.061835,1.676122,0.856263


fillna返回一个新对象，但你可以使用in-place来直接更改原有的数据：

In [24]:
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-1.169477,0.000000,0.000000
1,-1.000001,0.000000,0.000000
2,0.360180,0.000000,0.157768
3,0.908547,0.000000,-1.390020
4,-0.395955,0.531427,1.157834
5,-0.276360,-0.240133,1.478797
6,-0.061835,1.676122,0.856263


在使用fillna的时候，这种插入法同样能用于reindexing：

In [25]:
df = pd.DataFrame(np.random.randn(6, 3))
df

,0,1,2
0,1.172921,1.069823,0.566462
1,1.868118,-0.625675,-3.150870
2,0.115743,-0.202538,-0.783805
3,1.011211,-0.022856,0.127769
4,-0.474768,-0.177291,-0.258725
5,-1.796000,0.043277,1.367574


In [26]:
df.iloc[2:, 1] = NA
df

,0,1,2
0,1.172921,1.069823,0.566462
1,1.868118,-0.625675,-3.150870
2,0.115743,NaN,-0.783805
3,1.011211,NaN,0.127769
4,-0.474768,NaN,-0.258725
5,-1.796000,NaN,1.367574


In [27]:
df.iloc[4:, 2] = NA
df

,0,1,2
0,1.172921,1.069823,0.566462
1,1.868118,-0.625675,-3.150870
2,0.115743,NaN,-0.783805
3,1.011211,NaN,0.127769
4,-0.474768,NaN,NaN
5,-1.796000,NaN,NaN


In [28]:
df.fillna(method='ffill')

,0,1,2
0,1.172921,1.069823,0.566462
1,1.868118,-0.625675,-3.150870
2,0.115743,-0.625675,-0.783805
3,1.011211,-0.625675,0.127769
4,-0.474768,-0.625675,0.127769
5,-1.796000,-0.625675,0.127769


In [29]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,1.172921,1.069823,0.566462
1,1.868118,-0.625675,-3.150870
2,0.115743,-0.625675,-0.783805
3,1.011211,-0.625675,0.127769
4,-0.474768,NaN,0.127769
5,-1.796000,NaN,0.127769


使用fillna可以我们做一些颇有创造力的事情。比如，可以传入一个series的平均值或中位数：

In [30]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64